In [ ]:
# default_exp scraper

In [ ]:
%load_ext autoreload
%autoreload 2

# scraper

> Download all the PDF files from a particular web page.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import json
import os
import datetime
from pathlib import Path
from typing import Any

import pandas as pd
import numpy as np
import fitz

from rich import print
from rich.console import Console
from rich.table import Table

In [ ]:
%load_ext rich

- get all the links from a list of pages
- download all the PDF files from a list to a specific directory

In [ ]:
import os
import requests
import re
from urllib.parse import urlparse

from bs4 import BeautifulSoup

In [ ]:
#export
def add_scheme_and_domain(partial_url, main_source):
    """
    Given a partial url and a domain, return a full url.
    """
    scheme = urlparse(main_source).scheme
    hostname = urlparse(main_source).netloc
    base_url = scheme + "://" + hostname

    if partial_url[0] == "/":
        return base_url + partial_url
    else:
        return partial_url

In [ ]:
#export
def get_pdf_links(url):
    """
    Given a url, return a list of all pdf links on the page.
    """
    # Get the html of the page
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    # Find all the links on the page
    links = soup.find_all("a")
    # Find all the pdf links
    pdf_links = []
    for link in links:
        if link.get("href") is not None:
            if re.search(".pdf$", link.get("href")):
                pdf_links.append(add_scheme_and_domain(link.get("href"), url))
    return pdf_links

In [ ]:
#export
def download_pdf_files(pdf_links, destination="."):
    """
    Given a list of pdf links, download all the pdfs.
    """
    # create the destimation directory if it doesn't exist
    if not os.path.exists(destination):
        os.makedirs(destination)
    for link in pdf_links:
        r = requests.get(link)
        with open(destination + "/" + link.split("/")[-1], "wb") as f:
            f.write(r.content)

If you wish to download the PDF files from a particular webpage, use the following pattern:

```python
download_pdf_files(
    get_pdf_links("https://open.defense.gov/Transparency/FOIA.aspx"), "./test"
)
```